In [24]:
# creates caption/query GloVe-200 embeddings
import gensim
from gensim.models.keyedvectors import KeyedVectors
from cogworks_data.language import get_data_path
import re, string
from operator import add


path = get_data_path("glove.6B.200d.txt.w2v")
glove = KeyedVectors.load_word2vec_format(path, binary=False)

KeyboardInterrupt: 

In [2]:
import numpy as np

def embed_text(text):
    text = strip_punc(text).lower()
    text = text.split()
    print(text)
    
    embedded = np.zeros(200)
    for word in text:
        if word in glove:
            embedded += glove[word]
            
    return embedded

punc_regex = re.compile('[{}]'.format(re.escape(string.punctuation)))
def strip_punc(corpus):
    return punc_regex.sub('', corpus)


print(embed_text("Hello, world!").shape)

['hello', 'world']
(200,)


In [37]:
# myNN model for embedding image descriptors
import mygrad as mg
import mynn
from mynn.layers.dense import dense
from mygrad.nnet.losses.margin_ranking_loss import margin_ranking_loss
from mynn.optimizers.sgd import SGD
from mygrad.nnet.initializers import glorot_normal
from mygrad import no_autodiff
import pickle
from cogworks_data.language import get_data_path
from pathlib import Path
from Boueny import Database
import Boueny
import numpy as np

"""
(200, 512, 512) 
(200, 200, 200) (encoded)


cosine similarity between actual image and caption 
cosine similarity between confuser image and caption

loss function -> makes sure that actual image cosine similarity is always greater than confuser image
"""
def unzip(pairs):
    """
    "unzips" of groups of items into separate tuples.
    
    Example: pairs = [("a", 1), ("b", 2), ...] --> (("a", "b", ...), (1, 2, ...))
    
    Parameters
    ----------
    pairs : Iterable[Tuple[Any, ...]]
        An iterable of the form ((a0, b0, c0, ...), (a1, b1, c1, ...))
    
    Returns
    -------
    Tuple[Tuples[Any, ...], ...]
       A tuple containing the "unzipped" contents of `pairs`; i.e. 
       ((a0, a1, ...), (b0, b1, ...), (c0, c1), ...)
    """
    return np.array(tuple(zip(*pairs)))

def loss_function_accuracy(wtrueimg, wtruecap, wconfimg, margin): 
    print(wtrueimg.dtype, wtruecap.dtype, wconfimg.dtype)
    simtrue = mg.einsum("ni,ni -> n", wtruecap, wtrueimg)

    simconfuser = mg.einsum("ni,ni -> n", wtruecap, wconfimg)

    #returns both loss and accuracy, loss first
    
    accuracy = np.mean(simtrue.data  > simconfuser.data)  

    return (mg.mean(mg.maximum(0, margin - (simtrue - simconfuser))), accuracy)


class Autoencoder:
    def __init__(self, D_full, D_hidden):
        self.encode = dense(D_full, D_hidden, weight_initializer=glorot_normal, bias=False)
        
    
    def __call__(self, x):        
        return self.encode(x)
    
    
    @property
    def parameters(self):
        return self.encode.parameters

In [36]:
captions, trues, confusers = unzip(triplets[:4])
print(captions.shape)

(4, 200)


In [2]:
database = Database()
Boueny.save_database(database, "database.pkl")

loading GloVe
loading coco dataset
loading resnet18
Loading image url data
Loading captions
0.0 0.5035981555734426 1.1840233863435923 2.2690977555405216
generating caption embeddings


In [3]:
database = Boueny.load_database("database.pkl")

In [39]:
model = Autoencoder(512, 200)
optim = SGD(model.parameters, learning_rate=1e-3)

batch_size = 32 

for epoch_cnt in range(180):
    
    triplets = database.get_training_batch(batch_size)
    print(triplets[0][1].dtype)
    print(triplets[0][2].dtype)
    print(triplets[0][0].dtype)
    
    triplets = [(caption_id, model(true_id), model(confusor_id)) for caption_id, true_id, confusor_id in triplets]
    

    captions, trues, confusers = unzip(triplets)
    loss = loss_function_accuracy(captions, trues, confusers, 0.25)[0]

    loss.backward()
    optim.step()


    

float32
float32
float32
float64 float64 float64
float32
float32
float32
float32 float32 float32
float32
float32
float32
float32 float32 float32
float32
float32
float32
float32 float32 float32
float32
float32
float64
float64 float64 float64
float32
float32
float32
float32 float32 float32
float32
float32
float32
float64 float64 float64
float32
float32
float32
float32 float32 float32
float32
float32
float32
float64 float64 float64
float32
float32
float32
float32 float32 float32
float32
float32
float64
float64 float64 float64
float32
float32
float32
float32 float32 float32
float32
float32
float32
float64 float64 float64
float32
float32
float32
float32 float32 float32
float32
float32
float32
float64 float64 float64
float32
float32
float32
float32 float32 float32
float32
float32
float32
float64 float64 float64
float32
float32
float32
float64 float64 float64
float32
float32
float32
float32 float32 float32
float32
float32
float32
float64 float64 float64
float32
float32
float32
float32 float32 

float32
float32
float32
float32 float32 float32
float32
float32
float32
float32 float32 float32
float32
float32
float32
float64 float64 float64
float32
float32
float32
float32 float32 float32
float32
float32
float32
float32 float32 float32
float32
float32
float32
float32 float32 float32
float32
float32
float32
float32 float32 float32
float32
float32
float32
float32 float32 float32


NameError: name 'dvs' is not defined

In [40]:
with mg.no_autodiff:
    database.generate_image_embeddings(model)
    